In [1]:
# Standard libraries
import itertools as it
import os
import sys
import nibabel as nib
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import time
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from numpy.random import randint
import subprocess
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import subprocess
import torch
from skimage import measure

## we will use a tool called c3d for pre-processing and augmenting MRI images into 3D patches: 
# !wget https://sourceforge.net/projects/c3d/files/c3d/Experimental/c3d-1.1.0-Linux-gcc64.tar.gz 
# !tar -xvzf c3d-1.1.0-Linux-gcc64.tar.gz 

## add to path:
os.environ['PATH'] = os.environ['PATH'] + ':' + os.path.realpath('c3d-1.1.0-Linux-gcc64/bin/')

2023-01-01 17:09:48.262437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 17:09:48.415877: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /srv/software/itksnap/itksnap-3.8.2-alpha-20200410-Linux-gcc64-qt4/lib/snap-3.8.2-alpha:/srv/software/freesurfer/6.0.0/lib
2023-01-01 17:09:48.415915: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-01 17:09:50.042415: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64

In [2]:
def fid_voxel2world(fid_voxel, nii_affine):
    """Transform fiducials in voxel coordinates to world coordinates"""

    # Translation
    fid_world = fid_voxel.T + nii_affine[:3, 3:4]
    # Rotation
    fid_world = np.diag(np.dot(fid_world, nii_affine[:3, :3]))

    return fid_world.astype(float)

In [3]:
def fid_world2voxel(fid_world, nii_affine, resample_size=1, padding=None):
    """Transform fiducials in world coordinates to voxel coordinates

    Optionally, resample to match resampled image
    """

    # Translation
    fid_voxel = fid_world.T - nii_affine[:3, 3:4]
    # Rotation
    fid_voxel = np.dot(fid_voxel, np.linalg.inv(nii_affine[:3, :3]))

    # Round to nearest voxel
    fid_voxel = np.rint(np.diag(fid_voxel) * resample_size)

    if padding:
        fid_voxel = np.pad(fid_voxel, padding, mode="constant")

    return fid_voxel.astype(int)

In [4]:
def read_nii_metadata(nii_path):
    """Load in nifti data and header information and normalize MRI volume"""
    nii = nib.load(nii_path)
    nii_affine = nii.affine
    nii_data = nii.get_fdata()
    nii_data = (nii_data - nii_data.min())/ (nii_data.max() - nii_data.min())
    return nii_affine, nii_data

In [5]:
def get_fid(fcsv_path, fid_num):
    """Extract specific fiducial's spatial coordinates"""
    fcsv_df = pd.read_csv(fcsv_path, sep=",", header=2)
    return fcsv_df.loc[fid_num, ["x", "y", "z"]].to_numpy(dtype="single")

In [6]:
def seg_to_fcsv(weighted_centroids, fcsv_template, fcsv_output):
    # Read in fcsv template
    with open(fcsv_template, "r") as f:
        fcsv = [line.strip() for line in f]

    # Loop over fiducials
    for fid in range(1, 33):
        # Update fcsv, skipping header
        line_idx = fid + 2
        centroid_idx = fid - 1
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_x", str(weighted_centroids[centroid_idx][0])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_y", str(weighted_centroids[centroid_idx][1])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_z", str(weighted_centroids[centroid_idx][2])
        )

    # Write output fcsv
    with open(fcsv_output, "w") as f:
        f.write("\n".join(line for line in fcsv))

In [7]:
def Average(lst):
    return sum(lst) / len(lst)

In [8]:
 # Load image -- assumes correct bids spec 
DATASETS = 'HCP'
BASE_DIR_1 = f'/home/ROBARTS/ataha/graham/projects/ctb-akhanf/ataha24/5.afids_CNN/data/{DATASETS}/derivatives'
nii_path_mni = (f"/home/ROBARTS/ataha/graham/projects/ctb-akhanf/ataha24/5.afids_CNN/resources/templates/tpl-MNI152NLin2009cAsym_res-01_T1w.nii.gz")
mni_aff, mni_img = read_nii_metadata(nii_path_mni)
fcsv_path_mni = f"/home/ROBARTS/ataha/graham/projects/ctb-akhanf/ataha24/5.afids_CNN/resources/templates/tpl-MNI152NLin2009cAsym_res-01_T1w.fcsv"

# Get and compute new fiducial location
SUBJECT_IDS = [subject for subject in os.listdir(f'{BASE_DIR_1}/mni') if "sub-" in subject]
#fid_world = get_fid(fcsv_path, FID_NUM - 1)
#resampled_fid = fid_world2voxel(fid_world, aff, resample_size=SIZE, padding=PADDING)

In [9]:
for SUBJECT in SUBJECT_IDS: 
    print(SUBJECT)
    arrays_of_afids = np.empty((3,), dtype=float)
    for FID_NUM in range(1,33):
        model = 0 
        print(FID_NUM)
        model = tf.keras.models.load_model(f'{FID_NUM-1}')
        #extract afid coordinate from mni template
        PAD_FLAG = False
        PADDING = 0
        SIZE = 1  # Fixed for now but make variable 
        fid_world_mni = get_fid(fcsv_path_mni, FID_NUM - 1)
        resampled_fid_mni = fid_world2voxel(fid_world_mni, mni_aff, resample_size=SIZE, padding=PADDING)
        T1W_DIR= f"{BASE_DIR_1}/mni/{SUBJECT}/anat/{SUBJECT}_acq-MP2RAGE_space-MNI152NLin2009cAsym_res-1mm_desc-synthstripnorm_T1w.nii.gz"
        nii_path = T1W_DIR
        aff, img = read_nii_metadata(nii_path)
        x = 31
        x_1 = resampled_fid_mni[0] - x
        x_2 = resampled_fid_mni[0] + x+1
        y_1 = resampled_fid_mni[1] - x
        y_2 = resampled_fid_mni[1] + x+1
        z_1 = resampled_fid_mni[2] - x
        z_2 = resampled_fid_mni[2] + x+1
        pred = np.reshape(img[x_1:x_2,y_1:y_2,z_1:z_2], (1,63,63,63,1))
        distances = model.predict(pred)
        if distances.min() > 15: 
            print('minimum ED predicted is quite large. Placing mni coordinates as placeholder')
            AFID2 = fid_world_mni
            arrays_of_afids = np.vstack((arrays_of_afids, AFID2))
        else:     
            arr_dis = np.reshape(distances[0],(63,63,63))
            #print(arr_dis.min()) #tells how far in mm voxel is from afid
            new_pred = np.full((img.shape),100,dtype=float) #attached patch back in main image, other regions get 100mm prediction hard coded
            new_pred[x_1:x_2,y_1:y_2,z_1:z_2] = arr_dis #same as above it subs in the correct predictions at the patch
            transformed = np.exp(-0.5*new_pred) #transformed the prediction (i.e., distances from AFIDs into a probability e^-.5x
            #transformed[transformed >= thresh] = 1
            thresh = np.percentile(transformed, 99.999)
            transformed[transformed < thresh] = 0 #all voxels that are not in the top 99.999%ile are assigned as zero
            transformed = transformed*1000 #to prevent rounding errors due to astype 
            transformed = transformed.astype(int) 
            new = measure.regionprops(transformed) #uses scikit image to compute weighted centroid
            listx = []
            listy = []
            listz = []
            for centroids in new: #this extracts the average centroid for each voxel that has been assgined the same distance (ideally would be a sphere around ground truth hence "converge" on right answer)
                listx.append(centroids.centroid[0])
                listy.append(centroids.centroid[1])
                listz.append(centroids.centroid[2])
            arr_fid = np.array([Average(listx),Average(listy),Average(listz)])
            AFID = fid_voxel2world(arr_fid,aff)
            arrays_of_afids = np.vstack((arrays_of_afids, AFID))
            print(f'prediction for {FID_NUM} is {AFID}')
    seg_to_fcsv((arrays_of_afids[1:]), '/home/ROBARTS/ataha/graham/projects/ctb-akhanf/ataha24/5.afids_CNN/resources/afids_template.fcsv', f'/home/ROBARTS/ataha/graham/projects/ctb-akhanf/ataha24/5.afids_CNN/data/HCP/derivatives/mni/{SUBJECT}/anat/{SUBJECT}.fcsv')

sub-130013
1


2023-01-01 17:09:55.218758: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-01 17:09:55.218803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (AFI-CBS-H-12): /proc/driver/nvidia/version does not exist
2023-01-01 17:09:55.219144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 1s 633ms/step
prediction for 1 is [ 0.62447168  1.34270217 -2.83279093]
2
1/1 [==============================] - 1s 713ms/step
prediction for 2 is [  0.94087302 -24.20553666  -0.14744898]
3
1/1 [==============================] - 1s 806ms/step
prediction for 3 is [  0.71889276 -34.10311653  -4.32932636]
4
1/1 [==============================] - 1s 584ms/step
prediction for 4 is [-2.07570208e-02 -2.07600733e+01 -1.86794872e+01]
5
1/1 [==============================] - 1s 535ms/step
prediction for 5 is [  0.76412698 -13.93611111 -11.86730159]
6
1/1 [==============================] - 1s 593ms/step
prediction for 6 is [ 12.71203704 -26.57314815  -9.36898148]
7
1/1 [==============================] - 1s 610ms/step
prediction for 7 is [-12.91509434 -24.22327044  -8.03930818]
8
1/1 [==============================] - 1s 587ms/step
prediction for 8 is [ 12.24404762 -28.96934524 -18.30178571]
9
1/1 [==============================] - 1s 577ms/step
prediction fo

In [10]:
arrays_of_afids

array([[ 11.87875   ,  16.80708333,   9.65625   ],
       [  2.40634921,  -0.09707341,   7.28105159],
       [  1.15637973, -24.20689866,  10.43452381],
       [ -3.86904762, -37.7755102 ,  -0.3622449 ],
       [ -7.02879819, -20.86632653,  -7.95015117],
       [ -2.98519669, -15.53146998,   2.911853  ],
       [  5.73034188, -26.75641026,  -0.40299145],
       [-14.43      , -24.35055556,   6.78944444],
       [  1.22605442, -28.28095238, -12.18877551],
       [-16.80708736, -32.60761973, -10.43525107],
       [ -2.6865942 , -49.35688406,  10.92572464],
       [ -4.36369048,  -9.22251984,  -1.52748016],
       [ -1.42640873,  -9.81861111,  -2.71525794],
       [ -5.75894309,  -8.53780488,  -2.10853659],
       [ -2.14270833, -30.65138889,  12.75138889],
       [ 21.99207589,  -1.9749256 ,  26.3078869 ],
       [ -1.77252252,  -0.54279279,  34.70720721],
       [ 23.45555556, -26.36190476,  26.41746032],
       [ -8.75123387, -24.36296715,  36.21382458],
       [  7.41295671,  23.05910